# Import

In [65]:
import sys

sys.path.append("../")
import datetime
import json

import pandas as pd
import requests

from vook_db_v4.config import *

# Global

In [2]:
REQ_URL_CATE = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"

WANT_ITEMS = [
    "itemCode",
    "itemName",
    "itemPrice",
    "itemUrl",
]
PLATFORM_ID = 2

# Main

In [3]:
query = "リーバイス ヴィンテージ"

In [61]:
params = {
    "appid": ClientId,
    "output": "json",
    "query": query,
    "sort": "-price",
    "affiliate_id": aff_id,
    "affiliate_type": "vc",
    "results": 100,  # NOTE: 100個ずつしか取得できない。
}

start_num = 1
step = 100
max_products = 1000
l_df = []
for inc in range(0, max_products, step):
    params["start"] = start_num + inc
    df = pd.DataFrame(columns=WANT_ITEMS)
    res = requests.get(url=REQ_URL_CATE, params=params)
    res_cd = res.status_code
    if res_cd != 200:
        print(f"Bad request")
        break
    else:
        res = json.loads(res.text)
        if len(res["hits"]) == 0:
            print("If the number of returned items is 0, the loop ends.")
        print(f"Get Data")
        l_hit = []
        for h in res["hits"]:
            l_hit.append(
                (
                    h["index"],
                    h["name"],
                    h["price"],
                    h["url"],
                )
            )
        df = pd.DataFrame(l_hit, columns=WANT_ITEMS)
        l_df.append(df)

Get Data
Get Data
Get Data
Get Data
Get Data
Get Data
Get Data
Get Data
Get Data
Bad request


In [69]:
run_time = datetime.datetime.today().strftime("%Y%m%d_%H%M%S")
pd.concat(l_df, axis=0, ignore_index=True).to_csv(f"../data/output/{run_time}_for_issue_11.csv", index=False)

In [37]:
product_id_list = []
product_name_list = []
platform_id_list = []
for _, row in df.iterrows():
    product_id_list.append(hashlib.md5(str(row["itemCode"]).encode()).hexdigest())
    product_name_list.append(row["itemName"])
    platform_id_list.append(PLATFORM_ID)